In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import sklearn
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import confusion_matrix, mean_squared_error, r2_score 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest 
import matplotlib.pyplot as plt 
from sklearn.metrics import confusion_matrix 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, accuracy_score, recall_score 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor

# Read TSV file
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')
# Drop rows with missing values
train.dropna(inplace=True)
# 处理异常值
train = train[(train['revenue'] > 0) & (train['revenue'] < 1e6)]
# remove the stop word in ATM_Location_TYPE
stop_words = text.ENGLISH_STOP_WORDS.union(['Only', 'and'])
train['ATM_Location_TYPE'] = train['ATM_Location_TYPE'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
test['ATM_Location_TYPE'] = test['ATM_Location_TYPE'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# add feature
train['perhouse'] = train['Estimated_Number_of_Houses_in_1_KM_Radius'] / train['No_of_Other_ATMs_in_1_KM_radius']
test['perhouse'] = test['Estimated_Number_of_Houses_in_1_KM_Radius'] / test['No_of_Other_ATMs_in_1_KM_radius']
# Print first 5 rows of the DataFrame
train

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.10 from "/usr/local/bin/python3"
  * The NumPy version is: "1.23.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/jesse/Library/Python/3.10/lib/python/site-packages/numpy/core/_multiarray_umath.cpython-310-darwin.so, 0x0002): tried: '/Users/jesse/Library/Python/3.10/lib/python/site-packages/numpy/core/_multiarray_umath.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 

In [101]:

# split the ATM_Location_TYPE column
from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import PolynomialFeatures


train['ATM_Location_TYPE'] = train['ATM_Location_TYPE'].str.split(' ')
train = train.explode('ATM_Location_TYPE')
# Encode the string values to numeric values
# columns = ['ATM_Zone', 'ATM_Placement', 'ATM_TYPE', 'ATM_Location_TYPE', 'ATM_looks', 'ATM_Attached_to', 'Day_Type'] need to be encoded
cols_to_transform = [ 'ATM_Zone', 'ATM_Placement', 'ATM_TYPE', 'ATM_Location_TYPE', 'ATM_looks', 'ATM_Attached_to', 'Day_Type']

# define the LabelEncoder
le = LabelEncoder()
# apply the LabelEncoder to the columns
for col in cols_to_transform:
    le.fit(train[col].unique().tolist() + test[col].unique().tolist())
    
    # 将每个数据集中的列转换为整数
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

# Standardizing/normalizing data
#scaler = StandardScaler()
#train[['Estimated_Number_of_Houses_in_1_KM_Radius']] = scaler.fit_transform(train[['Estimated_Number_of_Houses_in_1_KM_Radius']])
#test[['Estimated_Number_of_Houses_in_1_KM_Radius']] = scaler.fit_transform(test[['Estimated_Number_of_Houses_in_1_KM_Radius']])

# PolynomialFeatures
#poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
#X_poly = poly.fit_transform(train.drop(['revenue'], axis=1))
#X_poly = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(train.drop(['revenue'], axis=1).columns))


# Divide the data into independent variables and dependent variables.
X = train.drop(['revenue'], axis=1)
Y = train['revenue']
# show the full result
pd.set_option('display.max_columns', None)
X

,Number_of_Shops_Around_ATM,ATM_Zone,No_of_Other_ATMs_in_1_KM_radius,Estimated_Number_of_Houses_in_1_KM_Radius,ATM_Placement,ATM_TYPE,ATM_Location_TYPE,ATM_looks,ATM_Attached_to,Average_Wait_Time,Day_Type,rating,perhouse
0,66,3,65,8450,0,3,7,1,0,3,3,4,130.000000
1,26,3,80,9600,0,3,7,1,0,3,3,3,120.000000
2,65,3,68,11250,0,2,7,1,0,3,3,4,165.441176
3,80,3,60,9550,0,2,7,1,0,3,3,4,159.166667
4,66,3,84,14260,0,2,7,1,0,4,3,4,169.761905
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150207,22,3,65,36500,0,2,0,0,0,4,3,3,561.538462
150207,22,3,65,36500,0,2,8,0,0,4,3,3,561.538462
150208,123,3,40,5664,0,2,7,1,0,2,3,5,141.600000
150209,64,3,86,11065,0,2,7,1,0,3,3,5,128.662791


In [102]:
# 向后消除法特征选择
while True:
    # 拟合模型
    model = sm.OLS(Y, sm.add_constant(X)).fit()
    # 计算每个特征的p值
    p_values = model.pvalues.iloc[1:]
    #print(p_values)
    # 选择p值最大的特征
    max_p_value = p_values.max()
    # 如果最大的p值大于0.05，则删除该特征
    if max_p_value > 0.05:
        feature_to_drop = p_values.idxmax()
        X = X.drop([feature_to_drop], axis=1)
        test = test.drop([feature_to_drop], axis=1)
    # 如果最大的p值小于等于0.05，停止迭代
    else:
        break
X


,Number_of_Shops_Around_ATM,ATM_Zone,No_of_Other_ATMs_in_1_KM_radius,ATM_Placement,ATM_TYPE,ATM_Location_TYPE,ATM_looks,ATM_Attached_to,Average_Wait_Time,Day_Type,rating,perhouse
0,66,3,65,0,3,7,1,0,3,3,4,130.000000
1,26,3,80,0,3,7,1,0,3,3,3,120.000000
2,65,3,68,0,2,7,1,0,3,3,4,165.441176
3,80,3,60,0,2,7,1,0,3,3,4,159.166667
4,66,3,84,0,2,7,1,0,4,3,4,169.761905
...,...,...,...,...,...,...,...,...,...,...,...,...
150207,22,3,65,0,2,0,0,0,4,3,3,561.538462
150207,22,3,65,0,2,8,0,0,4,3,3,561.538462
150208,123,3,40,0,2,7,1,0,2,3,5,141.600000
150209,64,3,86,0,2,7,1,0,3,3,5,128.662791


In [103]:

for i in range(20, 50, 2):

    model = RandomForestRegressor(n_estimators=180, max_depth=i, random_state=42)
    model.fit(X, Y)
    y_pred = model.predict(test.drop(['revenue'], axis=1))

    # calculate Pearson correlation coefficientt for the model
    corr, p_value = pearsonr(test['revenue'], y_pred)
    print(str(i) + ', ' + str(corr))

#y_pred = pd.DataFrame(y_pred, columns=['revenue'])
#y_pred.to_csv('z5320711.PART1.output.csv', index=False)
corr

20, 0.9973199487254787
22, 0.9978639025693536
24, 0.9979985953218216
26, 0.9980157412521014
28, 0.9980203043813893
30, 0.998020990144215
32, 0.9980197616625854
34, 0.9980189926229934
36, 0.9980190694245723
38, 0.9980190697430698
40, 0.9980190697430698
42, 0.9980190697430698
44, 0.9980190697430698
46, 0.9980190697430698
48, 0.9980190697430698


0.9980190697430698